# CatBoost Implementation

In [ ]:
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: catboost
    Found existing installation: catboost 1.2
    Uninstalling catboost-1.2:
      Successfully uninstalled catboost-1.2
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [1]:
!pip install numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.22.4-cp311-cp311-linux_x86_64.whl size=17328466 sha256=3cf4738e5a45e4c50aa484c7fd141bcc5352d3c55beb4870af54b22dc7255edf
  Stored in directory: /root/.cache/pip/wheels/8e/c0/7e/1583fa989ccf57e2059824c8783691f4927f2ce7b77cec9da2
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mizani 0.13.1 requires numpy>=1.23.5, but you have numpy 1.22.4 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.22.4 which i

In [1]:
#!/bin/bash
!curl -L -o ~/beth-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/katehighnam/beth-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.7M  100 39.7M    0     0  40.7M      0 --:--:-- --:--:-- --:--:--  118M


In [2]:
!unzip /root/beth-dataset.zip

Archive:  /root/beth-dataset.zip
replace labelled_2021may-ip-10-100-1-105-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-105-dns.csv  
replace labelled_2021may-ip-10-100-1-105.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-105.csv  
replace labelled_2021may-ip-10-100-1-186-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-186-dns.csv  
replace labelled_2021may-ip-10-100-1-186.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-186.csv  
replace labelled_2021may-ip-10-100-1-26-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-26-dns.csv  
replace labelled_2021may-ip-10-100-1-26.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-26.csv  
replace labelled_2021may-ip-10-100-1-4-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-i

In [3]:
import pandas as pd
import os

csv_files = [
    'labelled_testing_data.csv',
    'labelled_training_data.csv',
    'labelled_validation_data.csv'
]
combined_data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_data.to_csv('combined_dataset.csv', index=False)

print(f"Combined dataset shape: {combined_data.shape}")
print(combined_data.head())

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

Combined dataset shape: (1141078, 16)
    timestamp  processId  threadId  parentProcessId  userId  mountNamespace  \
0  129.050634        382       382                1     101      4026532232   
1  129.051238        379       379                1     100      4026532231   
2  129.051434          1         1                0       0      4026531840   
3  129.051481          1         1                0       0      4026531840   
4  129.051522          1         1                0       0      4026531840   

       processName         hostName  eventId           eventName  \
0  systemd-resolve  ip-10-100-1-217       41              socket   
1  systemd-network  ip-10-100-1-217       41              socket   
2          systemd  ip-10-100-1-217     1005  security_file_open   
3          systemd  ip-10-100-1-217      257              openat   
4          systemd  ip-10-100-1-217        5               fstat   

                                      stackAddresses  argsNum  returnValue  \


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('combined_dataset.csv')

def auto_classify_columns(data):
    categorical_columns = []
    numerical_columns = []

    for col in data.columns:
        if data[col].dtype == 'object':
            unique_vals = data[col].nunique()
            if unique_vals < 0.1 * len(data):
                categorical_columns.append(col)
            else:
                try:
                    data[col] = pd.to_numeric(data[col], errors='raise')
                    numerical_columns.append(col)
                except:
                    categorical_columns.append(col)
        else:
            numerical_columns.append(col)

    return categorical_columns, numerical_columns

categorical_columns, numerical_columns = auto_classify_columns(data)

print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py:14: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.sparse import csr_matrix, issparse


Categorical columns: ['processName', 'hostName', 'eventName', 'stackAddresses', 'args']
Numerical columns: ['timestamp', 'processId', 'threadId', 'parentProcessId', 'userId', 'mountNamespace', 'eventId', 'argsNum', 'returnValue', 'sus', 'evil']


In [5]:
label_encoder = LabelEncoder()
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

X = data.drop(columns=['evil', 'stackAddresses', 'args', 'argsNum', 'returnValue'])
y = data['evil']

X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Shape of X after scaling:", X_scaled.shape)

Shape of X after scaling: (1141078, 11)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [9]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


In [10]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    cat_features=[],
    verbose=100
)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on the test set
y_pred_cat = catboost_model.predict(X_test)
y_prob_cat = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the model using various metrics
accuracy_cat = accuracy_score(y_test, y_pred_cat)
precision_cat = precision_score(y_test, y_pred_cat)
recall_cat = recall_score(y_test, y_pred_cat)
f1_cat = f1_score(y_test, y_pred_cat)
cm_cat = confusion_matrix(y_test, y_pred_cat)

# Print evaluation metrics
print(f"CatBoost Model Evaluation:")
print(f"Accuracy: {accuracy_cat:.4f}")
print(f"Precision: {precision_cat:.4f}")
print(f"Recall: {recall_cat:.4f}")
print(f"F1 Score: {f1_cat:.4f}")


0:	learn: 0.3115333	total: 493ms	remaining: 8m 12s
100:	learn: 0.0000699	total: 28.8s	remaining: 4m 15s
200:	learn: 0.0000634	total: 46.8s	remaining: 3m 6s
300:	learn: 0.0000632	total: 1m 3s	remaining: 2m 27s
400:	learn: 0.0000632	total: 1m 19s	remaining: 1m 58s
500:	learn: 0.0000632	total: 1m 35s	remaining: 1m 35s
600:	learn: 0.0000632	total: 1m 52s	remaining: 1m 14s
700:	learn: 0.0000632	total: 2m 9s	remaining: 55.2s
800:	learn: 0.0000632	total: 2m 27s	remaining: 36.5s
900:	learn: 0.0000632	total: 2m 43s	remaining: 17.9s
999:	learn: 0.0000632	total: 2m 59s	remaining: 0us
CatBoost Model Evaluation:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [16]:
import joblib

joblib.dump(catboost_model, "catboost_model.pkl")
joblib.dump(y_test, 'y_test_catboost.joblib')
joblib.dump(y_prob_cat, 'y_prob_catboost.joblib')

print("Model and related artifacts saved successfully.")

Model and related artifacts saved successfully.
